In [ ]:
pip install imgbeddings

In [4]:
def VectorOfImage(strURL):
    import requests
    from PIL import Image
    url = strURL
    image = Image.open(requests.get(url, stream=True).raw)
    from imgbeddings import imgbeddings
    ibed = imgbeddings()
    embedding = ibed.to_embeddings(image)
    #query_vector=list(embedding.flatten())
    vector = embedding.flatten().tolist()
    return vector

In [5]:
import weaviate
import weaviate.classes as wvc
import requests
import json

In [6]:
client = weaviate.connect_to_wcs(
    cluster_url="",
    auth_credentials=weaviate.auth.AuthApiKey("")
    
)

In [7]:
client.is_ready()

True

In [ ]:
import weaviate.classes.config as wc
from weaviate.classes.config import Configure, Property, DataType
client.collections.create(
    name="imagesearch",
    properties=[
        wc.Property(name="imgsource", data_type=wc.DataType.TEXT),
        
    ],
    # Define the vectorizer module (none, as we will add our own vectors)
    vectorizer_config=wc.Configure.Vectorizer.none(),
    vector_index_config=Configure.VectorIndex.hnsw(),
)

In [ ]:

import time
t0 = time.time()
imgSources=[
'https://cdn.nba.com/manage/2021/08/michael-jordan-celebrates-archive.jpg',
'https://etimg.etb2bimg.com/photo/99593616.cms',
'https://asiasociety.org/sites/default/files/styles/1200w/public/1/150914_Manny_Pacquiao_Asia_Game_Changer.jpg',
'https://cdn.britannica.com/00/182400-050-BCFF616B/Manny-Pacquiao-2011.jpg'
]
DocumentSearch = client.collections.get("imagesearch")
with DocumentSearch.batch.dynamic() as batch:
    for i in range(0, 3): 
        properties = {"imgsource": imgSources[i]}
        vector=VectorOfImage(imgSources[i])
        batch.add_object(properties,vector=vector)
t1 = time.time()
total = t1-t0
print(total)  

In [15]:
def imageSearch(strURL):
    import weaviate.classes.query as wq
    query_text = strURL
 
    query_vector= VectorOfImage(strURL)
    
    DocumentSearch= client.collections.get("Imagesearch")
    response = DocumentSearch.query.near_vector(
    near_vector=query_vector,  # A list of floating point numbers
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
    )
    for o in response.objects:
        print(
        o.properties["imgsource"] 
    )  
        print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # P

In [23]:
imgSources=[
'https://etimg.etb2bimg.com/photo/99593616.cms',
'https://etimg.etb2bimg.com/photo/99593616.cms'

]
PlayerList =['MJ Michael Jordan','PacMan Manny Pacquio']
import time

t0 = time.time()

for i in range(0, 2): 
        print('i')
        print(PlayerList[i-1])
        imageSearch(imgSources[i-1])
t1 = time.time()
total = t1-t0
print('Time Taken')
print(total)  


i
PacMan Manny Pacquio
https://etimg.etb2bimg.com/photo/99593616.cms
Distance to query: 0.000

https://cdn.nba.com/manage/2021/08/michael-jordan-celebrates-archive.jpg
Distance to query: 0.128

https://asiasociety.org/sites/default/files/styles/1200w/public/1/150914_Manny_Pacquiao_Asia_Game_Changer.jpg
Distance to query: 0.141

i
MJ Michael Jordan
https://etimg.etb2bimg.com/photo/99593616.cms
Distance to query: 0.000

https://cdn.nba.com/manage/2021/08/michael-jordan-celebrates-archive.jpg
Distance to query: 0.128

https://asiasociety.org/sites/default/files/styles/1200w/public/1/150914_Manny_Pacquiao_Asia_Game_Changer.jpg
Distance to query: 0.141

Time Taken
5.572869539260864
